# Stories

## Primera historia: Análisis de ventas por categoría

In [1]:
import pandas as pd
import plotly.express as px

In [2]:
df = pd.read_parquet('../Data/cleaned.parquet')

In [3]:
df_radar = df.groupby(["Zona", "Canal"], as_index=False)["Monto de Venta"].sum()
fig_radar = px.line_polar(df_radar,
                          r="Monto de Venta", theta="Zona",
                          color="Canal", line_close=True,
                          template="plotly_dark")
fig_radar.show()

In [4]:
zona_sel = "Norte"
canal_sel = "Digital"
mes_a_num = {
    "Enero": 1, 
    "Febrero": 2, 
    "Marzo": 3, 
    "Abril": 4,
    "Mayo": 5
}


df_linea = df[(df["Zona"] == zona_sel) & (df["Canal"] == canal_sel)]
df_linea = df_linea.groupby(["Mes", "Producto"], as_index=False)["Monto de Venta"].sum()
df_linea["Mes"] = pd.Categorical(df_linea['Mes'], categories=mes_a_num.keys(), ordered=True)


fig_linea = px.line(df_linea,
                    x="Mes",
                    y="Monto de Venta",
                    color="Producto",
                    markers=True)
fig_linea.show()

## Segunda historia: Análisis de Asesores

In [5]:
df.columns

Index(['ID del Cliente', 'Zona', 'Canal', 'Producto', 'Monto de Venta',
       'Edad del Cliente', '¿Capacitado?', 'Inversión en Publicidad', 'Mes',
       'Asesor', '% Cumplimiento Objetivo', '% Clientes Nuevos',
       'Participación en Campañas', 'Antigüedad del Asesor (meses)'],
      dtype='object')

In [6]:
columnas_asesor = ['¿Capacitado?', 'Asesor', '% Cumplimiento Objetivo', 'Antigüedad del Asesor (meses)', 'Monto de Venta']

In [7]:
df[columnas_asesor].describe(include='all').T

,count,unique,top,freq,mean,std,min,25%,50%,75%,max
¿Capacitado?,1175,3,Sí,540,NaN,NaN,NaN,NaN,NaN,NaN,NaN
Asesor,1175,50,A28,35,NaN,NaN,NaN,NaN,NaN,NaN,NaN
% Cumplimiento Objetivo,1175.0,NaN,NaN,NaN,87.931387,19.406354,-5.0,78.89,89.18,99.68,139.15
Antigüedad del Asesor (meses),1175.0,NaN,NaN,NaN,30.067234,17.404392,1.0,15.0,30.0,46.0,59.0
Monto de Venta,1175.0,NaN,NaN,NaN,15283.934077,5025.131813,-1934.07,11843.315,15431.78,18695.555,32479.39


In [8]:
df_top10 = df.groupby('Asesor')['Monto de Venta'].sum().sort_values(ascending=False)
df_top10.head(5)

Asesor
A28    579220.50
A35    515216.10
A15    506319.93
A38    505694.82
A50    458560.85
Name: Monto de Venta, dtype: float64

In [9]:
df_top10_df = df_top10.head(10).reset_index()
fig1 = px.bar(
    df_top10_df,
    x='Monto de Venta',
    y='Asesor',
    orientation='h',
    title='Top 10 Asesores por Monto de Venta',
    labels={'Monto de Venta': 'Monto de Venta', 'Asesor': 'Asesor'},
    text='Monto de Venta'
)
fig1.update_traces(texttemplate='%{text:.2s}', textposition='outside')
fig1.show()

In [10]:
df_capacitado = df.groupby('¿Capacitado?')['Monto de Venta'].sum().reset_index()

fig_bar = px.bar(df_capacitado,
                 x='¿Capacitado?',
                 y='Monto de Venta',
                 title='Monto Total de Venta según Capacitación',
                 labels={'¿Capacitado?': '¿Capacitado?', 'Monto de Venta': 'Monto de Venta'})
fig_bar.show()


In [11]:
# Filtrar valores negativos en 'Monto de Venta'
df_scatter = df[df['Monto de Venta'] >= 0].copy()

fig3 = px.scatter(df_scatter,
                  x='Antigüedad del Asesor (meses)',
                  y='% Cumplimiento Objetivo',
                  color='¿Capacitado?',
                  size='Monto de Venta',
                  hover_name='Asesor',
                  title='Antigüedad vs Cumplimiento Objetivo',
                  labels={
                      'Antigüedad del Asesor (meses)': 'Antigüedad (meses)',
                      '% Cumplimiento Objetivo': '% Cumplimiento Objetivo',
                      '¿Capacitado?': '¿Capacitado?'
                  },
                  size_max=30)
fig3.show()

In [12]:
# Filtrar valores negativos en '% Cumplimiento Objetivo'
df_scatter_pos = df_scatter[df_scatter['% Cumplimiento Objetivo'] >= 0].copy()

fig3 = px.scatter(df_scatter_pos,
                  x='Antigüedad del Asesor (meses)',
                  y='Monto de Venta',
                  color='¿Capacitado?',
                  size='% Cumplimiento Objetivo',
                  hover_name='Asesor',
                  title='Antigüedad vs Cumplimiento Objetivo',
                  labels={
                      'Antigüedad del Asesor (meses)': 'Antigüedad (meses)',
                      '% Cumplimiento Objetivo': '% Cumplimiento Objetivo',
                      '¿Capacitado?': '¿Capacitado?'
                  },
                  size_max=20)
fig3.show()

## Tercera historia: Análisis de Clientes

In [13]:
df.columns

Index(['ID del Cliente', 'Zona', 'Canal', 'Producto', 'Monto de Venta',
       'Edad del Cliente', '¿Capacitado?', 'Inversión en Publicidad', 'Mes',
       'Asesor', '% Cumplimiento Objetivo', '% Clientes Nuevos',
       'Participación en Campañas', 'Antigüedad del Asesor (meses)'],
      dtype='object')

In [14]:
df['Participación en Campañas'].unique()

array(['Baja', 'Ninguna', 'Media', 'Alta', 'Desconocido'], dtype=object)

In [18]:
df_filtrado = df[df['Monto de Venta'] >= 0]

fig1 = px.scatter(
    df_filtrado,
    x="Inversión en Publicidad",
    y="% Clientes Nuevos",
    color="Participación en Campañas",
    size="Monto de Venta",
    hover_data=["Edad del Cliente", "Producto"],
    title="% Clientes Nuevos vs Inversión en Publicidad"
)
fig1.show()

In [20]:
fig2 = px.scatter(
    df,
    x="Edad del Cliente",
    y="Monto de Venta",
    color="Producto",
    size="% Clientes Nuevos",
    hover_data=["Inversión en Publicidad", "Participación en Campañas"],
    title="Edades de Clientes vs Monto de Venta"
)

fig2.show()